## Singular Value Decomposition
### What is the singular value decomposition?

The Singular-Value Decomposition, or SVD for short, is a matrix
decomposition method for reducing a matrix to its constituent parts
in order to make certain subsequent matrix calculations simpler. For
the case of simplicity we will focus on the SVD for real-valued matrices
and ignore the case for complex numbers.
    

$$A = U . \Sigma . V^{T}$$


Where $A$ is the real $n \times m$ matrix that we wish to decompose, $U$ is an
$m \times m$ matrix, $\Sigma$ represented by the uppercase Greek letter sigma)
is an $m \times n$ diagonal matrix, and $V^{T}$ is the $V$ transpose of an 
$n \times n$ matrix where $T$ is a superscript.

### Calculate Singular-Value Decomposition:

In [1]:
from numpy import array
from scipy.linalg import svd

# defining a matrix
A = array([
    [1, 2],
    [3, 4],
    [5, 6]
])

print(f"A: \n{A}\n")

# factorizing
U, s, V = svd(A)
print(f"U: \n{U}\n")
print(f"s: {s}\n")
print(f"V: \n{V}\n")

A: 
[[1 2]
 [3 4]
 [5 6]]

U: 
[[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]

s: [9.52551809 0.51430058]

V: 
[[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]



### Reconstruct Matrix:

The original matrix can be reconstructed from the $U$ , $\Sigma$ and $V^{T}$ elements.
The $U$, $s$, and $V$ elements returned from the $svd()$ cannot be multiplied directly.
The $s$ vector must be converted into a diagonal matrix using the $diag()$ function.
By default, this function will create a square matrix that is $m \times m$, relative to
our original matrix. This causes a problem as the size of the matrices do not fit the 
rules of matrix multiplication, where the number of columns in a matrix must match the
number of rows in the subsequent matrix. After creating the square $\Sigma$ diagonal matrix,
the sizes of the matrices are relative to the original $n \times m$ matrix that we are
decomposing, as follows:

$$U(m \times m).\Sigma(m \times m). V^{T}(n \times n)$$

Where, in fact, we require:

$$U(m \times m).\Sigma(m \times n). V^{T}(n \times n)$$

We can achieve this by creating a new $\Sigma$ matrix of all zero values that is $m \times n$ (e.g. more
rows) and populate the first $n \times n$ part of the matrix with the square diagonal matrix calculated
via $diag()$.

In [2]:
# reconstructing rectangular matrix fro svd

from numpy import array
from numpy import diag
from numpy import zeros
from scipy.linalg import svd

# defining a matrix
A = array([
    [1, 2],
    [3, 4],
    [5, 6]
])
print(f"A: \n{A}\n")

# factorizing
U, s, V = svd(A)

# creating m*n Sigma matrix
Sigma = zeros((A.shape[0], A.shape[1]))

# populate Sigma with n*n diagonal matrix
Sigma[:A.shape[1], :A.shape[1]] = diag(s)

# reconstructing matrix
B = U.dot(Sigma.dot(V))
print(f"B: \n{B}")

A: 
[[1 2]
 [3 4]
 [5 6]]

B: 
[[1. 2.]
 [3. 4.]
 [5. 6.]]


The above complication with the $\Sigma$ diagonal only exists with the case where m and n are
not equal. The diagonal matrix can be used directly when reconstructing a square matrix, as
follows:

In [3]:
from numpy import array
from numpy import diag
from scipy.linalg import svd

# defining a matrix
A = array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])
print(f"A: \n{A}\n")

# factorizing
U, s, V = svd(A)

# creating n*n Sigma matrix
Sigma = diag(s)

# reconstructing matrix
B = U.dot(Sigma.dot(V))
print(f"B: \n{B}")

A: 
[[1 2 3]
 [4 5 6]
 [7 8 9]]

B: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


## Pseudoinverse

The pseudoinverse is the generalization of the matrix inverse for square matrices to rectangular
matrices where the number of rows and columns are not equal. It is also called the Moore-Penrose
Inverse after two independent discoverers of the method or the Generalized Inverse.

The pseudoinverse is denoted as $A^{+}$, where $A$ is the matrix that is being inverted and $+$ is a
superscript. The pseudoinverse is calculated using the singular value decomposition of $A$:

$$A^{+} = V . D^{+} . U^{T}$$

The $D^{+}$ can be calculated by creating a diagonal matrix from $\Sigma$, calculating the reciprocal
of each non-zero element in $\Sigma$, and taking the transpose if the original matrix was rectangular.

$$ \Sigma = \begin{bmatrix} s_{1,1}  & 0 & 0 \\0 & s_{2,2} & 0 \\0 & 0 & s_{3,3}\end{bmatrix} $$

$$D^{+} = \begin{bmatrix} \frac{1}{s_{1,1}}  & 0 & 0 \\0 & \frac{1}{s_{2,2}} & 0 \\0 & 0 & \frac{1}{s_{3,3}}\end{bmatrix}$$

In [4]:
# pseudoinverse
from numpy.linalg import pinv

# defining an array
A = array([
    [0.1, 0.2],
    [0.3, 0.4],
    [0.5, 0.6],
    [0.7, 0.8]
])
print(f"A: \n{A}\n")

# calculating pseudoinverse
B = pinv(A)
print(f"B: \n{B}")

A: 
[[0.1 0.2]
 [0.3 0.4]
 [0.5 0.6]
 [0.7 0.8]]

B: 
[[-1.00000000e+01 -5.00000000e+00  1.42385628e-14  5.00000000e+00]
 [ 8.50000000e+00  4.50000000e+00  5.00000000e-01 -3.50000000e+00]]


We can calculate the pseudoinverse manually via the SVD and compare the results to the
$pinv()$ function. First we must calculate the $SVD$. Next we must calculate the reciprocal of
each value in the s array. Then the $s$ array can be transformed into a diagonal matrix with an
added row of zeros to make it rectangular. Finally, we can calculate the pseudoinverse from the
elements. The specific implementation is:

$$A^{+} = V^{T} . D^{T} . U^{T}$$

In [7]:
# pseudoinverse using svd
# defining an array
A = array([
    [0.1, 0.2],
    [0.3, 0.4],
    [0.5, 0.6],
    [0.7, 0.8]
])
print(f"A: \n{A}\n")

# factorizing
U, s, V = svd(A)

#reciprocals of s
d = 1.0 / s

# creating m*n D matrix
D = zeros(A.shape)

# populate D with n*n diagonal matrix
D[:A.shape[1], :A.shape[1]] = diag(d)

# calculating pseudoinverse
B = V.T.dot(D.T).dot(U.T)
print(f"B: \n{B}")

A: 
[[0.1 0.2]
 [0.3 0.4]
 [0.5 0.6]
 [0.7 0.8]]

B: 
[[-1.00000000e+01 -5.00000000e+00  1.42578328e-14  5.00000000e+00]
 [ 8.50000000e+00  4.50000000e+00  5.00000000e-01 -3.50000000e+00]]
